In [1]:
import modelskill as ms
import pandas as pd
import numpy as np

from mikeio1d.experimental import NetworkMapper

In [2]:
res1d_path = "../tests/testdata/network.res1d"
mapper = NetworkMapper(res1d_path)
network = mapper.map_network()

network.to_dataset()

<xarray.Dataset> Size: 231kB
Dimensions:     (time: 110, node: 259)
Coordinates:
  * time        (time) datetime64[ns] 880B 1994-08-07T16:35:00 ... 1994-08-07...
  * node        (node) int64 2kB 0 1 2 3 4 5 6 7 ... 252 253 254 255 256 257 258
Data variables:
    Discharge   (time, node) float32 114kB nan nan 5.72e-06 ... nan 0.01692 0.0
    WaterLevel  (time, node) float32 114kB 195.4 194.7 nan ... 188.5 nan nan

# Step 1: Create Network Model Result

Now we'll create a `NetworkModelResult` from the network dataset.

In [3]:
network_model = ms.NetworkModelResult(network.to_dataset(), item="WaterLevel")
network_model

<NetworkModelResult>: WaterLevel

# Step 2: Create Sample Node Observations

Let's create some synthetic node observations for demonstration purposes.

# Step 3: Extract Model Data at Observation Nodes

Now we'll extract the network model data at each observation location.

In [4]:
def dummy_series(t):
    return 1.0 + 0.5 * np.sin(np.arange(t) * 0.1) + 0.1 * np.random.randn(t)

model_time = network.to_dataset().time
n_sensors = 3
# Create DataFrame with observations for multiple nodes
obs_data = {}
for i in range(n_sensors):   
    obs_data[f'station_{i}'] = dummy_series(len(model_time))
    
obs_df = pd.DataFrame(obs_data, index=model_time)
obs_df.head()

,station_0,station_1,station_2
1994-08-07 16:35:00.000,0.926425,0.930964,0.997106
1994-08-07 16:36:01.870,1.052519,1.104768,1.058775
1994-08-07 16:37:07.560,1.060538,1.247758,1.058570
1994-08-07 16:38:55.828,1.248642,0.989696,1.208847
1994-08-07 16:39:55.828,1.233123,1.140802,1.032595


In [5]:
obs = []
nodes = [1, 3, 5]
for i, node in enumerate(nodes):
    obs_i = ms.NodeObservation(
        data=obs_df, 
        node=node,
        item=i
    )
    obs.append(obs_i)

# Step 4: Match and Analyze Results

Now let's match the network observations with the network model and compute skill metrics. The NetworkObservation will automatically be expanded to individual node observations during matching.

In [6]:
ms.match(obs, network_model).skill()

,n,bias,rmse,urmse,mae,cc,si,r2
observation,,,,,,,,
station_0,110,193.909936,193.911503,0.779407,193.909936,-0.434063,0.754674,-272915.446974
station_1,110,194.978542,194.979250,0.525355,194.978542,-0.363999,0.502146,-261857.307793
station_2,110,193.089926,193.091870,0.866424,193.089926,-0.197104,0.830038,-260353.933692
